In [1]:
# Create a workspce if it's not exited
from azureml.core import Workspace, Experiment, Run

ws = Workspace.create(name='AMLSLearnworkspace', 
                      subscription_id='485f9fa6-0221-4dbf-8074-a202426a29bf', 
                      resource_group='rgAMLSLearnworkspace',
                      create_resource_group=True,
                      location='East Asia'
                     )
print('Workspace created')

# Call a workspace if it's existed
#ws = Workspace.get(name='{name}',
#                   subscription_id='{azure-subscription-id}',
#                   resource_group='{resource-group-name}'
#                  )

#print('Done')


Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FHPTH4Y98 to authenticate.
Interactive authentication successfully completed.


Deploying KeyVault with name amlslearkeyvault59d10bc8.
Deploying StorageAccount with name amlslearstoragec5b27b52d.
Deploying AppInsights with name amlslearinsights1a904b7f.
Deployed AppInsights with name amlslearinsights1a904b7f. Took 11.74 seconds.
Deployed KeyVault with name amlslearkeyvault59d10bc8. Took 22.51 seconds.
Deployed StorageAccount with name amlslearstoragec5b27b52d. Took 30.54 seconds.
Deploying Workspace with name AMLSLearnworkspace.
Deployed Workspace with name AMLSLearnworkspace. Took 37.53 seconds.
Workspace created


In [2]:
# Load the diabetes dataset, a well-known built-in small dataset that comes with scikit-learn.
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

X, y = load_diabetes(return_X_y = True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

print('Done')

Done


In [3]:
experiment = Experiment(workspace = ws, name = "my-third-experiment")

print('Done')

Done


In [4]:
# Set up learninggoal for AutoML

from azureml.train.automl import AutoMLConfig
import logging

automl_config = AutoMLConfig(task = 'regression',
                  iteration_timeout_minutes = 10,
                  iterations = 3,
                  primary_metric = 'spearman_correlation',
                  n_cross_validations = 5,
                  debug_log = 'automl.log',
                  verbosity = logging.INFO,
                  X = X_train, 
                  y = y_train)

print('Done')

Done


In [5]:
# Start AutoML process
local_run = experiment.submit(automl_config, show_output = True)

Running on local machine
Parent Run ID: AutoML_ca4ba14a-7ab2-4612-8b8b-e2209c78a92e
Current status: DatasetCrossValidationSplit. Generating CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         0   StandardScalerWrapper RandomForest             0:01:20       0.6978    0.6978
         1   MinMaxScaler RandomForest                      0:00:52       0.6807    0.6978
         2   StandardScalerWrapper ExtremeRandomTrees   

In [6]:
# Show the results from each model
from azureml.widgets import RunDetails
RunDetails(local_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [7]:
# Pick the best model with the highest correlation
best_run, fitted_model = local_run.get_output()

In [8]:
# Evaluate the best model
y_pred_train = fitted_model.predict(X_train)
y_residual_train = y_train - y_pred_train
y_pred_test = fitted_model.predict(X_test)
y_residual_test = y_test - y_pred_test

In [9]:
print(y_residual_test)

[ 1.03332717e+02 -2.03024847e+01 -5.30951821e+01 -5.38521775e+01
  3.59862278e+01  3.53218191e+01  8.16326108e+01  1.03381571e+01
  1.69414974e+01 -1.09398779e+02  8.93676372e+01 -1.86245227e+00
  1.46897858e+00 -2.28272125e+01  1.46850451e+01 -5.12853590e+01
 -5.29226180e+01 -1.02626341e+01  3.25492419e+00 -6.30705079e+01
 -2.52255293e+00  1.34855589e+02 -6.40730789e+01  1.59717062e+01
 -1.30825174e+02 -5.56124652e+01 -1.42180856e+01  8.92875705e+00
  2.43817763e+01  2.48060947e+01  5.33771850e+01  8.54439034e+01
  5.52793025e+01  6.07163538e+01  5.86030747e+01  1.01503297e+02
  2.46734523e-02  1.08468834e+02 -5.28711335e+01 -1.66798363e+01
 -1.79909079e+01  1.83960746e+01 -4.55620353e+01  3.66920247e+01
  5.63817599e+01  4.85142626e+00  9.43828779e+01  6.11071096e+01
 -3.83062602e+01  4.46090938e+01  1.59244615e+01  4.95291997e+01
 -2.45310247e+01 -5.12288986e+01 -3.23428712e+01 -1.39724122e+01
 -7.17529383e+01 -3.19126501e+01  8.75862442e+01  5.45860043e+01
 -1.64600052e+02 -2.79833